# LLM standard library

Given our diagram's ability to use generic functions, we can create a standard library of functions that are useful for LLMs.

In [ ]:
# |default_exp chat

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

import pytest

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export
from stringdale.core import get_git_root, load_env, checkLogs,  json_render,json_undeclared_vars,disk_cache
import openai 
from pydantic import BaseModel, create_model
from typing import Optional, Dict, Any, List, Union
import json
import re
from parse import parse
from pathlib import Path
from enum import Enum
from pydantic import BaseModel
import logging
import os
from stringdale.core import semaphore_decorator


In [ ]:
#| export
load_env()
logger = logging.getLogger(__name__)


In [ ]:
#| export
from typing import (
    Callable, 
    Type, 
    Optional, 
    List, 
    Dict, 
    Any, 
    Union,
    Literal,
    get_type_hints
)
from pydantic import (
    BaseModel, 
    Field, 
    create_model,
    ConfigDict
)
import inspect
import sys
import sqlmodel
from typing import Optional
# TODO make chat outputschema also get typing (like bool)

In [ ]:
import nest_asyncio

In [ ]:
nest_asyncio.apply()

In [ ]:
#| export
import instructor
from openai import OpenAI,AsyncOpenAI
from anthropic import Anthropic, AsyncAnthropic
from pydantic import BaseModel
from singleton_decorator import singleton
import asyncio


## Completion functions


### Openai

In [ ]:
#| export
## OpenAI clients
@singleton
def json_openai_client():
    return instructor.from_openai(AsyncOpenAI(),mode=instructor.Mode.JSON)
@singleton
def raw_openai_client():
    return AsyncOpenAI()

In [ ]:
#| export
def mcp_tools_to_openai_kwargs(mcp_tools):
    """
    Convert MCP tools to OpenAI's tools parameter format.
    
    Args:
        mcp_tools: List of MCP Tool objects (can be empty)
        
    Returns:
        Dict with 'tools' key containing OpenAI-formatted tools, or empty dict if no tools
    """
    if not mcp_tools:
        return {}
    
    openai_tools = [{
        "type": "function",
        "function": {
            "name": tool.name,
            "description": tool.description,
            "parameters": tool.inputSchema
        }
    } for tool in mcp_tools]
    
    return {"tools": openai_tools}


In [ ]:

#| export
@disk_cache.cache(ignore=['response_model'])
async def complete_open_ai(model, messages, response_model=None, response_schema=None, mode='json', seed=42, **kwargs):
    """
    OpenAI-specific completion handler.
    Chooses between clients based on mode and runs the completion.
    """
    if mode == 'json':
        client = json_openai_client()
        response, completion = await client.chat.completions.create_with_completion(
            model=model,
            messages=messages,
            response_model=response_model,
            seed=seed,
            **kwargs
        )
        usage = {
            "input_tokens": completion.usage.prompt_tokens,
            "output_tokens": completion.usage.completion_tokens
        }
        return response.model_dump_json(), usage


    elif mode == 'mcp_tools' or mode == 'raw':
        # Unified path: handle both raw (no tools) and mcp_tools cases
        client = raw_openai_client()
        
        # Get MCP tools from kwargs (default to empty list for 'raw' mode)
        mcp_tools = kwargs.pop('mcp_tools', [])
        
        # Convert MCP tools to OpenAI format
        tools_kwargs = mcp_tools_to_openai_kwargs(mcp_tools)
        
        # Filter out unsupported parameters if needed
        openai_kwargs = {k: v for k, v in kwargs.items() 
                        if k not in ['print_prompt']}
        
        # Call OpenAI API
        completion = await client.chat.completions.create(
            model=model,
            messages=messages,
            seed=seed,
            **tools_kwargs,  # Includes tools if any
            **openai_kwargs
        )
        
        # Extract usage information
        usage = {
            "input_tokens": completion.usage.prompt_tokens,
            "output_tokens": completion.usage.completion_tokens
        }
        
        # Parse response into unified format
        message = completion.choices[0].message
        result = {
            "text": message.content if message.content else None
        }
        
        # Extract tool calls if any
        if message.tool_calls:
            result["tool_calls"] = []
            for tool_call in message.tool_calls:
                result["tool_calls"].append({
                    "name": tool_call.function.name,
                    "input": json.loads(tool_call.function.arguments),
                    "id": tool_call.id
                })
        
        # For backward compatibility with 'raw' mode: if no tools and no tool_calls, return just text string
        if mode == 'raw' and not result.get("tool_calls") and not mcp_tools:
            return result["text"], usage
        
        return result, usage
    
    else:
        raise ValueError(f"Invalid mode: {mode}")

In [ ]:
from fastmcp import Client

In [ ]:

weather_path = str(get_git_root()/"stringdale/mcp_weather_server.py")
config = {
  "mcpServers": {
    "weather": {
      "command": "python",
      "args": [weather_path]
    }
  }
}
mcp_client = Client(config)
async with mcp_client:
    mcp_tools = await mcp_client.list_tools()

example_messages = [
    {"role": "user", "content": "What is the weather like in Seattle? "}
]


In [ ]:
result, usage = await complete_open_ai(
    model="gpt-4o-mini",
    messages=example_messages,
    mode="mcp_tools",
    mcp_tools=mcp_tools
)
print("Result:", result)
print("Usage:", usage)

Result: {'text': None, 'tool_calls': [{'name': 'get_forecast', 'input': {'latitude': 47.6062, 'longitude': -122.3321}, 'id': 'call_maBN72l2QQq6JLc1cqjbjmUr'}]}
Usage: {'input_tokens': 135, 'output_tokens': 26}


### Anthropic


In [ ]:
## Anthropic clients
@singleton
def raw_anthropic_client():
    return AsyncAnthropic()
    
# Patching the Anthropics client with the instructor for enhanced capabilities
@singleton
def json_anthropic_client():
    anthropic_client = instructor.from_anthropic(
    AsyncAnthropic(),
    mode=instructor.Mode.ANTHROPIC_JSON
)
    return anthropic_client


In [ ]:
#| export
def mcp_tools_to_anthropic_kwargs(mcp_tools):
    """
    Convert MCP tools to Anthropic's tools parameter format.
    
    Args:
        mcp_tools: List of MCP Tool objects (can be empty)
        
    Returns:
        Dict with 'tools' key containing Anthropic-formatted tools, or empty dict if no tools
    """
    if not mcp_tools:
        return {}
    
    anthropic_tools = [{
        "name": tool.name,
        "description": tool.description,
        "input_schema": tool.inputSchema
    } for tool in mcp_tools]
    
    return {"tools": anthropic_tools}

In [ ]:
#| export
@disk_cache.cache(ignore=['response_model'])
async def complete_anthropic(model, messages, response_model=None, response_schema=None, mode='json', seed=42, **kwargs):
    """
    Anthropic-specific completion handler.
    Chooses between clients based on mode and runs the completion.
    """
    def _extract_usage(usage_obj):
        """Extract usage information from Anthropic usage object"""
        return {
            "input_tokens": getattr(usage_obj, "input_tokens", 0),
            "output_tokens": getattr(usage_obj, "output_tokens", 0)
        }
    
    max_tokens = kwargs.get('max_tokens', 1024)
    # Filter out unsupported Anthropic parameters
    # Anthropic doesn't support 'stop' or 'seed' parameters
    anthropic_kwargs = {k: v for k, v in kwargs.items() 
                        if k not in ['stop', 'seed', 'print_prompt', 'mcp_tools']}
    
    if mode == 'json':
        client = json_anthropic_client()
        response, completion = await client.chat.completions.create_with_completion(
            model=model,
            messages=messages,
            response_model=response_model,
            max_tokens=max_tokens,
            **anthropic_kwargs
        )
        return response.model_dump_json(), _extract_usage(completion.usage)

    elif mode == 'mcp_tools' or mode == 'raw':
        # Unified path: handle both raw (no tools) and mcp_tools cases
        client = raw_anthropic_client()
        
        # Get MCP tools from kwargs (default to empty list for 'raw' mode)
        mcp_tools = kwargs.pop('mcp_tools', [])
        
        # Convert MCP tools to Anthropic format
        tools_kwargs = mcp_tools_to_anthropic_kwargs(mcp_tools)
        
        # Extract system message (first system message), rest as regular messages
        system = None
        stripped = []
        for m in messages:
            role = m.get("role")
            content = m.get("content", "")
            if role == "system" and system is None:
                system = content
            else:
                stripped.append({"role": role, "content": content})
        
        # Call Anthropic API
        response = await client.messages.create(
            model=model,
            system=system if system else "",  # Anthropic allows empty string for system
            messages=stripped,
            max_tokens=max_tokens,
            **tools_kwargs,  # Includes tools if any
            **anthropic_kwargs
        )
        
        # Parse response content blocks
        text_parts = []
        tool_use_blocks = []
        
        for block in (response.content or []):
            if getattr(block, "type", None) == "text":
                text_parts.append(block.text)
            elif getattr(block, "type", None) == "tool_use":
                # Extract tool use information
                tool_use_blocks.append({
                    "name": block.name,
                    "input": block.input,  # Already a dict, no need to parse JSON
                    "id": getattr(block, "id", None)  # Include id if available
                })
        
        # Extract usage information
        usage = _extract_usage(response.usage)
        
        # Return consistent structure: always a dict with 'text' and optionally 'tool_calls'
        result = {
            "text": "\n".join(text_parts) if text_parts else None
        }
        if tool_use_blocks:
            result["tool_calls"] = tool_use_blocks
        
        # For backward compatibility with 'raw' mode: if no tools and no tool_calls, return just text string
        if mode == 'raw' and not tool_use_blocks and not mcp_tools:
            return result["text"], usage
        
        return result, usage
    
    else:
        raise ValueError(f"Invalid mode: {mode}")

Testing mcp tools mode for anthropic

In [ ]:
result, usage = await complete_anthropic(
    model="claude-3-haiku-20240307",
    messages=example_messages,
    mode="mcp_tools",
    mcp_tools=mcp_tools
)
print("Result:", result)
print("Usage:", usage)


Result: {'text': 'Okay, let me check the weather forecast for Seattle:', 'tool_calls': [{'name': 'get_forecast', 'input': {'latitude': 47.6062, 'longitude': -122.3321}, 'id': 'toolu_019XLKX5aQS59Ck58unkCH5b'}]}
Usage: {'input_tokens': 501, 'output_tokens': 90}


In [ ]:

tool_calls = result.get('tool_calls', [])
if not tool_calls:
    raise ValueError("No tool calls found in the result.")
tool_name = tool_calls[0]['name']
tool_args = tool_calls[0]['input']  # Already a dict, no need to parse JSON
print(f"Tool Used: {tool_name}, Arguments: {tool_args}")

# Execute the tool called by the LLM
async with mcp_client:
    tool_response = await mcp_client.call_tool(tool_name, tool_args)
    tool_response_text = tool_response.content[0].text


Tool Used: get_forecast, Arguments: {'latitude': 47.6062, 'longitude': -122.3321}


In [ ]:
print(tool_response_text)


This Afternoon:
Temperature: 42°F
Wind: 12 mph SSW
Forecast: A chance of rain and snow showers before 2pm, then a slight chance of rain and snow showers between 2pm and 3pm, then a chance of rain and snow showers. Mostly cloudy, with a high near 42. South southwest wind around 12 mph, with gusts as high as 22 mph. Chance of precipitation is 50%. New rainfall amounts between a tenth and quarter of an inch possible.

---

Tonight:
Temperature: 36°F
Wind: 6 to 10 mph SSW
Forecast: Rain and snow showers likely before 4am, then a chance of rain and snow showers. Mostly cloudy, with a low around 36. South southwest wind 6 to 10 mph. Chance of precipitation is 60%. New rainfall amounts less than a tenth of an inch possible.

---

Thursday:
Temperature: 44°F
Wind: 6 to 9 mph S
Forecast: Rain and snow showers likely before 4pm, then a chance of rain. Mostly cloudy, with a high near 44. South wind 6 to 9 mph. Chance of precipitation is 70%. New rainfall amounts less than a tenth of an inch poss

### Here we're setting up  everything we need to get tools and session for mcp.
We're going to update chat to work with them

In [ ]:

weather_path = str(get_git_root()/"stringdale/mcp_weather_server.py")
config = {
  "mcpServers": {
    "weather": {
      "command": "python",
      "args": [weather_path]
    }
  }
}
mcp_client = Client(config)
async with mcp_client:
    mcp_tools = await mcp_client.list_tools()

example_messages = [
    {"role": "user", "content": "What is the weather like in Seattle? "}
]


In [ ]:
#| export
async def complete_raw(model, messages, response_model=None, response_schema=None, mode='json', seed=42, provider=None, **kwargs):
    """
    This function is used to complete a chat completion with instructor without having basemodels as input or output.
    used for disk caching of results.
    
    Chooses what provider and calls a provider specific caller by model name with provider override.
    
    Args:
        model: Model name (e.g., 'gpt-4', 'claude-3-haiku-20240307')
        messages: List of message dicts
        response_model: Pydantic model for structured output
        response_schema: Schema for response (computed if response_model is provided)
        mode: Completion mode ('raw', 'json', 'tools')
        seed: Random seed
        provider: Optional provider override ('openai', 'anthropic'). If None, inferred from model name.
        **kwargs: Additional arguments passed to the provider.
                  Important provider-specific requirements:
                  - Anthropic: max_tokens is REQUIRED (e.g., max_tokens=1024)
    """
    # Determine provider from model name or use override
    if provider is None:
        if model.startswith(('gpt-', 'o1-')):
            provider = 'openai'
        elif model.startswith('claude-'):
            provider = 'anthropic'
        else:
            # Default to OpenAI for unknown models
            provider = 'openai'
    
    # Route to provider-specific function
    if provider == 'openai':
        return await complete_open_ai(
            model=model,
            messages=messages,
            response_model=response_model,
            response_schema=response_schema,
            mode=mode,
            seed=seed,
            **kwargs
        )
    elif provider == 'anthropic':
        return await complete_anthropic(
            model=model,
            messages=messages,
            response_model=response_model,
            response_schema=response_schema,
            mode=mode,
            **kwargs
        )
    else:
        raise ValueError(f"Unknown provider: {provider}")

async def complete(model, messages, response_model, mode='json', print_prompt=False, **kwargs):
    # Compute schema if response model provided
    if isinstance(response_model, type) and issubclass(response_model, BaseModel):
        response_schema = response_model.model_json_schema()
    else:
        response_schema = str(response_model)
    if print_prompt:
        print(messages)
    response, usage = await complete_raw(
        model=model,
        messages=messages,
        response_model=response_model,
        response_schema=response_schema,
        mode=mode,
        **kwargs
    )
    if mode == 'raw' or mode == 'mcp_tools':
        return response, usage
    else:
        # Check if response is already a model instance (from instructor)
        if isinstance(response, BaseModel):
            return response, usage
        else:
            return response_model.model_validate_json(response), usage

In [ ]:

class UserExtract(BaseModel):
    name: str
    age: int


user, usage = await complete(
    model="gpt-3.5-turbo",
    response_model=UserExtract,
    messages=[
        {"role": "user", "content": "Extract jason is 25 years old"},
    ],
)

user,usage

(UserExtract(name='Extract jason', age=25),
 {'input_tokens': 147, 'output_tokens': 18})

In [ ]:
user, usage = await complete(
    model="claude-sonnet-4-5",
    response_model=UserExtract,
    messages=[
        {"role": "user", "content": "Extract jason is 25 years old"},
    ],
)

user,usage

(UserExtract(name='jason', age=25),
 {'input_tokens': 326, 'output_tokens': 1110})

## Completion types

### Simpler answer

In [ ]:
#| export
async def answer_question(model,messages,**api_kwargs):
    res,usage = await complete(model,messages,response_model=None,mode='raw',**api_kwargs)
    return res,usage


In [ ]:
answer = await answer_question("gpt-3.5-turbo",[{"role":"user","content":"What is the capital of France?"}])
print(answer)

('The capital of France is Paris.', {'input_tokens': 14, 'output_tokens': 7})


In [ ]:
# Test: Integration test with answer_question - anthropic
answer, usage = await answer_question(
    model="claude-sonnet-4-5",
    messages=[{"role": "user", "content": "What is the capital of France?"}],
)

assert isinstance(answer, str)
assert "Paris" in answer or "paris" in answer.lower()
assert isinstance(usage, dict)

### Choice

In [ ]:
#| export
async def choose(model,messages,choices,**api_kwargs):
    class Choice(BaseModel):
        choice: Literal[tuple(choices)]
    res,usage = await complete(model,messages,Choice,**api_kwargs)
    return res.choice,usage


In [ ]:
await choose("gpt-3.5-turbo",[{"role":"user","content":"What is the capital of the country France?"}],["PARIS", "HILTON"],print_prompt=True)


[{'role': 'user', 'content': 'What is the capital of the country France?'}]


('PARIS', {'input_tokens': 140, 'output_tokens': 10})

In [ ]:
await choose("claude-sonnet-4-5",[{"role":"user","content":"What is the capital of the country France?"}],["PARIS", "HILTON"],print_prompt=True)

[{'role': 'user', 'content': 'What is the capital of the country France?'}]


('PARIS', {'input_tokens': 154, 'output_tokens': 92})

### Multi choice

In [ ]:
#| export
async def choose_many(model,messages,choices,**api_kwargs):
    class Choice(BaseModel):
        choice: Literal[tuple(choices)]

    class Choices(BaseModel):
        choices: List[Choice]   
    res,usage = await complete(model,messages,Choices,**api_kwargs)
    return [c.choice for c in res.choices],usage


In [ ]:

new_answer = await choose_many("gpt-3.5-turbo",
    messages=[{"role":"user","content":"what parameters did i pass in? my name is jason and i am 25 years old"}],
    choices=["Age", "Name","City"],print_prompt=True)

print(new_answer)

[{'role': 'user', 'content': 'what parameters did i pass in? my name is jason and i am 25 years old'}]
(['Name', 'Age'], {'input_tokens': 235, 'output_tokens': 31})


In [ ]:
# calude
await choose_many("claude-sonnet-4-5",
    messages=[{"role":"user","content":"what parameters did i pass in? my name is jason and i am 25 years old"}],
    choices=["Age", "Name","City"],print_prompt=True)

[{'role': 'user', 'content': 'what parameters did i pass in? my name is jason and i am 25 years old'}]


(['Name', 'Age'], {'input_tokens': 272, 'output_tokens': 105})

### Structured output

In [ ]:
#| export
def clean_model(model: Type[sqlmodel.SQLModel], name: Optional[str] = None) -> Type[BaseModel]:
    """Convert an SQLModel to a Pydantic BaseModel.
    used to clean up the output for the LLM
    Args:
        model: SQLModel class to convert
        name: Optional name for the new model class
        
    Returns:
        A Pydantic BaseModel class with the same fields
    """
    # Get field definitions from the SQLModel
    fields = {}
    for field_name, field in model.model_fields.items():
        fields[field_name] = (field.annotation, field)
    
    # Create new model name if not provided
    model_name = name or f"{model.__name__}Schema"
    
    # Create and return new Pydantic model
    return create_model(model_name, **fields)

In [ ]:
#| export
async def structured_output(model,messages,output_schema,as_json=False,**api_kwargs):

    is_sqlmodel = isinstance(output_schema,type) and issubclass(output_schema,sqlmodel.SQLModel)
    if is_sqlmodel:
        clean_schema = clean_model(output_schema)
    else:
        clean_schema = output_schema

    res,usage = await complete(model,messages,clean_schema,**api_kwargs)

    if is_sqlmodel:
        res = output_schema(**res.model_dump())
    if as_json:
        res = res.model_dump()
    return res,usage


In [ ]:
class UserExtract(BaseModel):
    name: str
    age: int



res,usage = await structured_output("gpt-3.5-turbo",
    [{"role":"user","content":"what parameters did i pass in? my name is Jason and i am 25 years old"}],
    UserExtract)
assert res == UserExtract(name="Jason",age=25), res
print(res)
print(usage)




name='Jason' age=25
{'input_tokens': 157, 'output_tokens': 16}


In [ ]:
# claude
res,usage = await structured_output("claude-sonnet-4-5",
    [{"role":"user","content":"what parameters did i pass in? my name is Jason and i am 25 years old"}],
    UserExtract)

print(res)
print(usage)



name='Jason' age=25
{'input_tokens': 174, 'output_tokens': 132}


In [ ]:

import sqlite3
from sqlalchemy.engine import create_engine

In [ ]:

#| export
class User(sqlmodel.SQLModel, table=False):
    id: Optional[int] = sqlmodel.Field(default=None, primary_key=True)
    name: Optional[str] = sqlmodel.Field(default=None)
    age: Optional[int] = sqlmodel.Field(default=None)
    email: Optional[str] = sqlmodel.Field(default=None)



In [ ]:

res,usage = await structured_output("gpt-3.5-turbo",
    [{"role":"user","content":"my name is jyson, my age is 25, my id is 1"}],
    User)
print(res)


id=1 name='jyson' age=25 email=None


In [ ]:
# anthropic
anth_res,usage = await structured_output("claude-sonnet-4-5",
    [{"role":"user","content":"my name is jyson, my age is 25, my id is 1"}],
    User)
print(res)
assert anth_res==res

id=1 name='jyson' age=25 email=None


## The main chat class

In [ ]:
#| export
from copy import deepcopy,copy
from pprint import pformat,pprint


In [ ]:
#| export

class Chat:
    """A Chat objects the renders a prompt and calls an LLM. Currently supporting openai models.
    
    Args:
        model: OpenAI model name
        messages: List of message dicts, must have at least a role and content field
        output_schema: Optional schema for structured output
        as_json: Optional boolean to return the response as a json object
        tools: Optional dictionary of tool names and functions that the LLM can decide to call. Causes the content of the response
            to be a dict of the form {'name':tool_name,'input':tool_input_dict}
        call_function: if tools are provided, whether to call the function and save the output in the output field of the response's content
        choices: Optional List of choices for multi-choice questions
        multi_choice: if choices are provided, whether to choose multiple items from the list
        seed: Optional seed for random number generation
        stop: Optional string or list of strings where the model should stop generating
        save_history: Optional boolean to save the history of the chat between calls
        append_output: Optional, whether to append the output of the chat to history automatically, default False
        init_messages: Optional list of messages that are always prepended to messages.
            Useful for supplying additional messages during calls.
            Can have template variables that are fed during initialization only.
            If save_history is True, the init messages are added to the history.
        **kwargs: Keyword arguments to interpolate into the messages
    """
    def __init__(self,
        model: Optional[str] = None,
        messages: Optional[List[Dict[str, str]]] = None, 
        output_schema: Optional[BaseModel] = None,
        as_json: Optional[bool] = False,
        tools: Optional[Dict[str,Callable]] = None,
        call_function: Optional[bool] = False,
        choices: Optional[Enum] = None,
        multi_choice: Optional[bool] = False,
        seed: Optional[int] = 42,
        stop: Optional[Union[str, List[str]]] = None,
        log_prompt: bool = False,
        save_history: bool = False,
        append_output: bool = False,
        init_messages: Optional[List[Dict[str, str]]] = None,
        **kwargs):

        self.model = model
        self.messages = deepcopy(messages)
        self.output_schema = output_schema
        self.as_json = as_json
        self.tools = tools
        self.call_function = call_function
        self.choices = choices
        self.multi_choice = multi_choice
        self.seed = seed
        self.stop = stop
        self.log_prompt = log_prompt
        self.baked_kwargs = kwargs
        self.save_history = save_history
        self.append_output = append_output
        
        if init_messages is None:
            init_messages = []
        self.init_messages = json_render(init_messages,context=kwargs)
    
        self.reset()

    def reset(self):
        """Resets state of Chat"""
        if self.save_history:
            self.history = []
            self.history.extend(self.init_messages)
        else:
            self.history = None
    
    def dump_state(self):
        """dumps the node state"""
        return self.history

    def load_state(self,state_object):
        """loads node state"""
        self.history = state_object

    def __copy__(self):
        chat_copy = Chat(
            model=self.model,
            messages=self.messages,
            output_schema=self.output_schema,
            as_json=self.as_json,
            tools=self.tools,
            call_function=self.call_function,
            choices=self.choices,
            multi_choice=self.multi_choice,
            seed=self.seed,
            stop=self.stop,
            log_prompt=self.log_prompt,
            save_history=self.save_history,
            append_output=self.append_output,
            init_messages=self.init_messages,
            **self.baked_kwargs
        )
        return chat_copy

    async def __call__(self, **kwargs) -> Dict[str, Any]:
        """Format prompt with kwargs and call OpenAI chat.
        Init parameters such as output_schema, tools, choices, seed, stop, as well as template variables
        can be set or overridden by kwargs
        
        Args:
            **kwargs: Values for format string placeholders
            
        Returns:
            a dictionary with the following keys:
            - role (str): Always "assistant"
            - content: the llm response.
            - meta (dict): Usage statistics including input and output tokens
        """
        model = kwargs.get("model", self.model)
        messages = kwargs.get("messages", self.messages)
        output_schema = kwargs.get("output_schema", self.output_schema)
        as_json = kwargs.get("as_json", self.as_json)
        tools = kwargs.get("tools", self.tools)
        call_function = kwargs.get("call_function", self.call_function)
        choices = kwargs.get("choices", self.choices)
        multi_choice = kwargs.get("multi_choice", self.multi_choice)
        seed = kwargs.get("seed", self.seed)
        stop = kwargs.get("stop", self.stop)

        if model is None:
            raise ValueError("model is required but not provided")
        if messages is None:
            raise ValueError("messages is required but not provided")

        prompt_kwargs = {**self.baked_kwargs, **kwargs}

        required_kwargs = json_undeclared_vars(messages)
        if not required_kwargs <= set(prompt_kwargs):
            missing = required_kwargs - set(prompt_kwargs)
            raise ValueError(f"Missing required kwargs: {missing}")

        formatted_messages = json_render(messages, context=prompt_kwargs)

        if self.save_history:
            self.history.extend(formatted_messages)
            formatted_messages = self.history
        else:
            formatted_messages = self.init_messages + formatted_messages

        if self.log_prompt:
            logger.warning(f'calling llm with model={model} and prompt:\n'
                        f'messages={pformat(formatted_messages)}\n'
                        )

        completion_kwargs = {
            'model':model,
            'messages':formatted_messages,
            'seed':seed,
            'stop':stop,
            'print_prompt':prompt_kwargs.get('print_prompt',False),
        }

        if choices:
            if multi_choice:
                res,usage = await choose_many(choices=choices,**completion_kwargs)
            else:
                res,usage = await choose(choices=choices,**completion_kwargs)
        elif output_schema:
            res,usage = await structured_output(output_schema=output_schema,as_json=as_json,**completion_kwargs)
        elif tools:
            res, usage = await complete(
                mode='mcp_tools',
                mcp_tools=tools,
                response_model=None,
                **completion_kwargs
            )
        else:
            res,usage = await answer_question(**completion_kwargs)

        response = {
            'role':'assistant',
            'content':res,
            'meta':usage
        }
        if self.save_history and self.append_output:
            self.history.append(response)
        return response
    
    def __str__(self) -> str:
        """String representation showing required keys, model, and output schema."""
        parts = [f"Chat(model='{self.model}'"]
        
        if self.messages:
            required_keys = json_undeclared_vars(self.messages) - set(self.baked_kwargs.keys())
            parts.append(f"required_keys={required_keys}")
            
        if self.output_schema:
            parts.append(f"output_schema={self.output_schema.__name__}")
        
        if self.tools:
            # tools is now a list of Tool objects, extract their names
            tool_names = [tool.name for tool in self.tools]
            parts.append(f"""tools={",".join(tool_names)}""")
            #parts.append(f"""tools={",".join(self.tools.keys())}""")
        if self.call_function:
            parts.append(f"call_function={self.call_function}")
            
        # if self.choices:
        #     parts.append(f"choices={self.choices}")
        # if self.multi_choice:
        #     parts.append(f"multi_choice={self.multi_choice}")
            
        if self.seed:
            parts.append(f"seed={self.seed}")
            
        if self.stop:
            parts.append(f"stop={self.stop}")

        if self.save_history:
            parts.append(f"save_history={self.save_history}")
            
        return ", ".join(parts) + ")"
    
    def metadata(self) -> Dict[str, Any]:
        """Return metadata about the chat."""
        meta =  {
            'model':self.model,
            'messages':self.messages,
            'output_schema':self.output_schema,
            'tools':self.tools,
            'call_function':self.call_function,
            'choices':self.choices,
            'multi_choice':self.multi_choice,
            'seed':self.seed,
            'stop':self.stop,
            
        }
        if self.save_history:
            meta['history'] = self.history
        meta = {k:v for k,v in meta.items() if v is not None and v is not False}
        return meta

    def __repr__(self) -> str:
        """Same as string representation."""
        return self.__str__()

For a cheatsheet on how to use jinja templates, see [this link](https://devhints.io/jinja)

## Chat Tests

### Basic

In [ ]:
chat_with_history = Chat(model="gpt-3.5-turbo", 
                        save_history=True,
                        append_output=True,
                        init_messages=[{"role":"system","content":"You are a helpful {{role}}"}],
                        messages=[{"role": "user", "content": "Hi, im {{name}}, answer me: {{text}}"}],
                        role = 'AI overlord',
                        name = 'ernio',
                        log_prompt=True
                        )
res = await chat_with_history(text="What is the capital of France?",print_prompt=True)
res

calling llm with model=gpt-3.5-turbo and prompt:
messages=[{'content': 'You are a helpful AI overlord', 'role': 'system'},
 {'content': 'Hi, im ernio, answer me: What is the capital of France?',
  'role': 'user'}]



[{'role': 'system', 'content': 'You are a helpful AI overlord'}, {'role': 'user', 'content': 'Hi, im ernio, answer me: What is the capital of France?'}]


{'role': 'assistant',
 'content': 'Hello Ernio! The capital of France is Paris.',
 'meta': {'input_tokens': 34, 'output_tokens': 11}}

In [ ]:
chat_with_history.history

[{'role': 'system', 'content': 'You are a helpful AI overlord'},
 {'role': 'user',
  'content': 'Hi, im ernio, answer me: What is the capital of France?'},
 {'role': 'assistant',
  'content': 'Hello Ernio! The capital of France is Paris.',
  'meta': {'input_tokens': 34, 'output_tokens': 11}}]

In [ ]:
for i in range(3):
    res = await chat_with_history(messages=[{'role':'user','content':'Question: what is obamas age to the power of 2?'}])
    chat_with_history.reset()
    # print(chat_with_history.history)
    print(res)
    print('='*100)

calling llm with model=gpt-3.5-turbo and prompt:
messages=[{'content': 'You are a helpful AI overlord', 'role': 'system'},
 {'content': 'Hi, im ernio, answer me: What is the capital of France?',
  'role': 'user'},
 {'content': 'Hello Ernio! The capital of France is Paris.',
  'meta': {'input_tokens': 34, 'output_tokens': 11},
  'role': 'assistant'},
 {'content': 'Question: what is obamas age to the power of 2?', 'role': 'user'}]

calling llm with model=gpt-3.5-turbo and prompt:
messages=[{'content': 'You are a helpful AI overlord', 'role': 'system'},
 {'content': 'Question: what is obamas age to the power of 2?', 'role': 'user'}]

calling llm with model=gpt-3.5-turbo and prompt:
messages=[{'content': 'You are a helpful AI overlord', 'role': 'system'},
 {'content': 'Question: what is obamas age to the power of 2?', 'role': 'user'}]



{'role': 'assistant', 'content': "To calculate President Obama's age to the power of 2, we first need to know his current age. As of 2021, Barack Obama was born on August 4, 1961. Therefore, if we assume it's 2021, his age would be 60.\n\nNow let's calculate his age to the power of 2:\n\nAge squared = 60^2 = 60 * 60 = 3600\n\nSo, President Obama's age to the power of 2 is 3600.", 'meta': {'input_tokens': 67, 'output_tokens': 107}}
{'role': 'assistant', 'content': "Obama was born on August 4, 1961. To find his age squared, we must first calculate his current age.\n\nAs of September 2021, Obama is 60 years old. Therefore, his age squared would be 60^2, which is equal to 3600.\n\nSo, Obama's age to the power of 2 is 3600.", 'meta': {'input_tokens': 32, 'output_tokens': 76}}
{'role': 'assistant', 'content': "Obama was born on August 4, 1961. To find his age squared, we must first calculate his current age.\n\nAs of September 2021, Obama is 60 years old. Therefore, his age squared would be 

In [ ]:
assert len(chat_with_history.history) == 1 , len(chat_with_history.history)
chat_with_history.history

[{'role': 'system', 'content': 'You are a helpful AI overlord'}]

In [ ]:
res = await chat_with_history(text="And what is the closest city to it?",print_prompt=True)
res

calling llm with model=gpt-3.5-turbo and prompt:
messages=[{'content': 'You are a helpful AI overlord', 'role': 'system'},
 {'content': 'Hi, im ernio, answer me: And what is the closest city to it?',
  'role': 'user'}]



[{'role': 'system', 'content': 'You are a helpful AI overlord'}, {'role': 'user', 'content': 'Hi, im ernio, answer me: And what is the closest city to it?'}]


{'role': 'assistant',
 'content': "Hello, Ernio. I'm here to help. Could you please provide more context or details so that I can assist you better?",
 'meta': {'input_tokens': 36, 'output_tokens': 27}}

In [ ]:
assert len(chat_with_history.history)==3

### Choices

In [ ]:
messages=[
        {"role": "system", "content": "Given a sentence, classify it into one of these topics: science, history, technology, or arts. Choose the single most relevant topic."},
        {"role": "user", "content": "{{text}}"}
    ]
    
topic_classifier = Chat(
    model="gpt-4o-mini",
    messages=messages,
    choices = ['science', 'history', 'technology', 'arts'],
    seed=42,
    log_prompt=True
)
topic_classifier


Chat(model='gpt-4o-mini', required_keys={'text'}, seed=42)

In [ ]:
renaisance_topic = await topic_classifier(text="WWII was a global conflict that lasted from 1939 to 1945.")
astroid_topic = await topic_classifier(text="The asteroid belt is a region of space between the orbits of Mars and Jupiter.")

assert renaisance_topic['content'] == 'history',renaisance_topic
assert astroid_topic['content'] == 'science',astroid_topic



calling llm with model=gpt-4o-mini and prompt:
messages=[{'content': 'Given a sentence, classify it into one of these topics: science, '
             'history, technology, or arts. Choose the single most relevant '
             'topic.',
  'role': 'system'},
 {'content': 'WWII was a global conflict that lasted from 1939 to 1945.',
  'role': 'user'}]

calling llm with model=gpt-4o-mini and prompt:
messages=[{'content': 'Given a sentence, classify it into one of these topics: science, '
             'history, technology, or arts. Choose the single most relevant '
             'topic.',
  'role': 'system'},
 {'content': 'The asteroid belt is a region of space between the orbits of '
             'Mars and Jupiter.',
  'role': 'user'}]



### Structured output

In [ ]:
class Person(BaseModel):
    first_name: str
    last_name: str
    date_of_birth: int

prompted_llm = Chat(model="gpt-4o-mini", messages=
    [   
        {"role": "user", "content": "how old am i? {{name}}, {{age}} years old"},
        {"role": "assistant", "content": "Iam {{model_name}}, You are {{name}}, {{age}} years old"}
    ],
     output_schema=Person)
prompted_llm


Chat(model='gpt-4o-mini', required_keys={'age', 'model_name', 'name'}, output_schema=Person, seed=42)

In [ ]:
baked_llm = Chat(model="gpt-4o-mini", messages=
    [
        {"role": "user", "content": "how old am i? {{name}}, {{age}} years old"},
        {"role": "assistant", "content": "Iam {{model_name}}, You are {{name}}, {{age}} years old"}
    ],
    output_schema=Person, model_name="gpt-4o-mini", age=30)
baked_llm


Chat(model='gpt-4o-mini', required_keys={'name'}, output_schema=Person, seed=42)

In [ ]:
res = await prompted_llm(model_name="gpt-4o-mini", age=30,name="Dean")
assert res['content'] == Person(first_name='Dean', last_name='', date_of_birth=1993)
res

{'role': 'assistant',
 'content': Person(first_name='Dean', last_name='', date_of_birth=1993),
 'meta': {'input_tokens': 206, 'output_tokens': 26}}

In [ ]:
res = await baked_llm(name="Dean")
assert res['content'] == Person(first_name='Dean', last_name='', date_of_birth=1993)
res

{'role': 'assistant',
 'content': Person(first_name='Dean', last_name='', date_of_birth=1993),
 'meta': {'input_tokens': 206, 'output_tokens': 26}}

In [ ]:
res = await baked_llm(name="Dean",as_json=True)
assert res['content'] == {"first_name": "Dean", "last_name": "", "date_of_birth": 1993} 
res

{'role': 'assistant',
 'content': {'first_name': 'Dean', 'last_name': '', 'date_of_birth': 1993},
 'meta': {'input_tokens': 206, 'output_tokens': 26}}

In [ ]:
course_chooser = Chat(model="gpt-4o-mini",
    messages=[{"role": "user", "content": "{{text}}"}],
    choices=["science","genocide", "history", "defence against the dark arts", "arts"],
    multi_choice=True)
res = await course_chooser(text="choose everything that is not genocide")
assert not 'genocide' in res['content'] , res
res

{'role': 'assistant',
 'content': ['science', 'history', 'defence against the dark arts', 'arts'],
 'meta': {'input_tokens': 236, 'output_tokens': 58}}

### MCP Tools

In [ ]:

weather_path = str(get_git_root()/"stringdale/mcp_weather_server.py")
config = {
  "mcpServers": {
    "weather": {
      "command": "python",
      "args": [weather_path]
    }
  }
}
mcp_client = Client(config)
async with mcp_client:
    mcp_tools = await mcp_client.list_tools()



In [ ]:
# Chooses tool when needs to
user_input = "What is the weather like in Seattle?"

chat_with_mcp_tools = Chat(
    model="gpt-4o-mini",
    tools=mcp_tools
)
res = await chat_with_mcp_tools(messages=[{"role": "user", "content": user_input}])
assert res["role"] == "assistant", res
content = res["content"]
assert content["text"] is None, content
assert "tool_calls" in content and isinstance(content["tool_calls"], list), content
tool_call = content["tool_calls"][0]
assert tool_call["name"] == "get_forecast", tool_call
assert tool_call["input"] == {"latitude": 47.6062, "longitude": -122.3321}, tool_call
tool_call

{'name': 'get_forecast',
 'input': {'latitude': 47.6062, 'longitude': -122.3321},
 'id': 'call_lOu4arEdGpMT8BneWnlKC9cY'}

In [ ]:
# Doesn't choose tool when doesn't need to
user_input = "What is a dog?"

chat_with_mcp_tools = Chat(
    model="gpt-4o-mini",
    tools=mcp_tools
)
res = await chat_with_mcp_tools(messages=[{"role": "user", "content": user_input}])
assert res["role"] == "assistant", res
content = res["content"]
assert content.get("text") is not None, content
assert "tool_calls" not in content or content["tool_calls"] is None, content
content

{'text': 'A dog is a domesticated mammal and a subspecies of the gray wolf (Canis lupus). Dogs belong to the family Canidae and are known scientifically as Canis lupus familiaris. They have been bred for thousands of years for various traits, which has led to a wide variety of breeds that differ in size, shape, temperament, and purpose.\n\nDogs are often referred to as "man\'s best friend" due to their long-standing companionship with humans. They are known for their loyalty, intelligence, and ability to be trained for various tasks, including hunting, herding, guarding, and serving as service animals. Dogs are also popular as family pets, providing companionship and affection.\n\nIn addition to their roles as pets and working animals, dogs communicate through various vocalizations (barking, whining, growling) and body language. Their senses, particularly smell and hearing, are highly developed, making them excellent at tasks such as search and rescue, detection of drugs or explosives,

### Init messages without saving history

In [ ]:
chat_with_init_messages = Chat(model="gpt-4o-mini",
    init_messages=[{"role": "system", "content": "You are an unhelpful assistant. Whenever asked to help, you say no."}],
)
res = await chat_with_init_messages(messages=[{"role": "user", "content": "What is the capital of {{country}}?"}],country="France")
res
assert 'no' in res['content'].lower(),res

## Image to Text

In [ ]:
#| export
@disk_cache.cache
async def image_to_text(path:str,model:str="gpt-4o-mini",url=False):
    """
    This function takes an image (either from a local file path or URL) and uses OpenAI's
    vision model to generate a detailed description of the image contents. The results are
    cached using disk_cache to avoid redundant API calls.
        
    Args:
        path (str): Path to the image file or URL of the image
        model (str, optional): OpenAI model to use for image analysis. Defaults to "gpt-4o-mini".
        url (bool, optional): Whether the path is a URL. Defaults to False.
        
    Returns:
        dict: A dictionary containing:
            - role (str): Always "assistant"
            - content (str): Detailed description of the image
            - meta (dict): Usage statistics including input and output tokens
    
    """
    if url:
        image = instructor.Image.from_url(path)
    else:
        image = instructor.Image.from_path(path)

    class ImageAnalyzer(BaseModel):
        description:str

    res,usage = await complete(
        model=model,
        messages=[{"role":"user","content":[
            "What is in this image, please describe it in detail\n",
            image,
            "\n"
        ]}],
        response_model=ImageAnalyzer,
    )
    return {
        'role':'assistant',
        'content':res.description,
        'meta':usage
    }


In [ ]:
from textwrap import wrap

In [ ]:
res= await image_to_text(get_git_root()/"sample_data/fox.jpeg")

assert 'fox' in res['content']
print('\n'.join(wrap(res['content'],width=100)))
res


The image features a close-up of a fox's face, showcasing its distinct features. The fox has a bushy
coat with a mix of reddish-brown and cream colors, with a characteristic white patch on its chin.
Its ears are pointed and alert, standing upright, with a lighter fur lining. The eyes are sharp and
focused, exhibiting an amber hue that contrasts with its fur. The background appears softly blurred,
adding emphasis to the fox's facial details and enhancing the warm tones of its fur.


{'role': 'assistant',
 'content': "The image features a close-up of a fox's face, showcasing its distinct features. The fox has a bushy coat with a mix of reddish-brown and cream colors, with a characteristic white patch on its chin. Its ears are pointed and alert, standing upright, with a lighter fur lining. The eyes are sharp and focused, exhibiting an amber hue that contrasts with its fur. The background appears softly blurred, adding emphasis to the fox's facial details and enhancing the warm tones of its fur.",
 'meta': {'input_tokens': 8627, 'output_tokens': 107}}

In [ ]:
from textwrap import wrap


In [ ]:
image_url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/3/30/Vulpes_vulpes_ssp_fulvus.jpg/800px-Vulpes_vulpes_ssp_fulvus.jpg'
res= await image_to_text(image_url,url=True)

assert 'fox' in res['content']
print('\n'.join(wrap(res['content'],width=100)))
res


The image features a red fox standing in a snowy landscape. The fox has a vibrant orange-brown fur
coat with a white underbelly, and its tail is bushy with a dark tip. Its ears are pointed and stand
upright, and it has a sharp, keen expression with bright, amber-colored eyes. The snowy ground
contrasts with the color of its fur, and there are blurred trees in the background, which are also
covered with snow, indicating a wintry environment.


{'role': 'assistant',
 'content': 'The image features a red fox standing in a snowy landscape. The fox has a vibrant orange-brown fur coat with a white underbelly, and its tail is bushy with a dark tip. Its ears are pointed and stand upright, and it has a sharp, keen expression with bright, amber-colored eyes. The snowy ground contrasts with the color of its fur, and there are blurred trees in the background, which are also covered with snow, indicating a wintry environment.',
 'meta': {'input_tokens': 25628, 'output_tokens': 103}}

## Speech to text

In [ ]:
#| export
from instructor.multimodal import Audio
import openai

In [ ]:
#| export

@disk_cache.cache
async def speech_to_text(audio_path: str, model: str = "whisper-1") -> Dict[str,str]:
    """Extract text from an audio file using OpenAI's Whisper model.
    
    Args:
        audio_path (str): Path to the audio file
        model (str, optional): OpenAI model to use. Defaults to "whisper-1".
    
    Returns:
        dict: A dictionary containing:  
            - role (str): Always "assistant"
            - content (str): Transcribed text from the audio
    """
    client = raw_client()

    with open(audio_path, "rb") as audio_file:
        response = await client.audio.transcriptions.create(
            model=model,
            file=audio_file
        )
    
    res =  {
        'role':'assistant',
        'content':response.text,
    }
    
    return res

In [ ]:
res = await speech_to_text(get_git_root()/"sample_data/happy_speech.wav")
assert res['content'] == "Look at this, my hands are standing up in my arms, I'm giving myself goosebumps." , res
res

{'role': 'assistant',
 'content': "Look at this, my hands are standing up in my arms, I'm giving myself goosebumps."}

## Export

In [ ]:
# |hide
import nbdev

nbdev.nbdev_export()